In [38]:
!pip install gender_guesser

In [66]:
import pandas as pd

In [67]:
novostroy_df = pd.read_csv("reviews.csv")
zkh_df = pd.read_csv("forum-zhk.csv")

In [68]:
novostroy_df

,Name,Date,Review,Rating
0,Алина,2025-01-31,К выбору квартиры подошла основательно. Сначал...,5
1,Михаил,2025-01-28,Переехали мы год назад в новостройку от компан...,1
2,Ольга Николина,2025-01-24,Живу пол года в жилом комплексе Пригород Лесно...,5
3,Евгения Жовниренко,2025-01-21,Категорически не рекомендую самолёт.Все отличн...,1
4,Мarina,2025-01-20,"Остановилась на несколько ЖК от Самолёта, но в...",5
...,...,...,...,...
390,Евгений Filipov,2018-07-09,"Все нормально , но если какая то проблема в от...",1
391,Пользователь,2018-07-05,"Проблему по ЖК ОСТАФЬЕВО, решили оперативно!",5
392,Пользователь,2018-05-08,Здравствуйте! Купили квартиру в Люберцах. Срок...,3
393,Полина Полина Гришина,2018-05-07,Задумалась о покупке квартиры в ЖК спутник . В...,3


In [69]:
zkh_df = zkh_df.dropna()
zkh_df = zkh_df[~zkh_df["Review"].str.startswith("Siteinfo: IP-адреса отзывов из одной", na=False)]
zkh_df["Date"] = pd.to_datetime(zkh_df["Date"], format="%d.%m.%Y %H:%M").dt.strftime("%Y-%m-%d")
zkh_df["Rating"] = zkh_df["Rating"].apply(int)
combined_df = pd.concat([novostroy_df, zkh_df], ignore_index=True)

In [70]:
combined_df.to_csv("United_data.csv")

### Длинна комментария

In [71]:
combined_df["Review_Length"] = combined_df["Review"].str.len()

### Добавить столбец с полом пользователя

In [72]:
# !pip install natasha
# !pip install setuptools

In [73]:
from natasha import (
    NamesExtractor,
    MorphVocab
)

In [77]:
# Создаём морфологический словарь
morph_vocab = MorphVocab()

# Передаём его в конструктор NamesExtractor
names_extractor = NamesExtractor(morph_vocab)

def get_first_name(full_name: str) -> str:
    clean_name = full_name.replace('.', ' ').replace(',', ' ').strip()
    
    # Преобразуем результат NamesExtractor в список
    matches = list(names_extractor(clean_name))
    
    if not matches:
        # Фоллбэк: берем первое слово
        return clean_name.split()[0]
    
    parsed_name = matches[0].fact.first
    if parsed_name is None:
        parsed_name = clean_name.split()[0]
    
    return parsed_name

def determine_gender(name: str) -> str:
    """
    Определяем пол на основе имени, используя простые правила и данные Natasha.
    Возможные возвращаемые значения: "male", "female", "unknown".
    """
    # Пытаемся разобрать как PersonName
    # Сам Natasha для пола напрямую полную проверку не даёт, 
    # но можно по косвенным признакам определять,
    # либо использовать pymorphy2 (см. ниже).
    
    # Для примера покажем простой набор эвристик:
    # - Если имя заканчивается на "а", "я", "ия" и т.п. -> female (с оговорками)
    # - Иначе предполагаем male.
    # Вы можете усложнить этот блок любым нужным вам способом, 
    # подключить дополнительную словарную базу, сделать исключения и т.д.
    
    # Переведём в нижний регистр для удобства проверок
    name_lower = name.lower()
    
    # Несколько самых частых женских окончаний
    female_endings = ("а", "я", "ия", "иа", "на", "ся", "ля")
    
    # Допустим, если слово оканчивается на любое из этих окончаний, считаем женским именем
    if any(name_lower.endswith(end) for end in female_endings):
        return "female"
    
    # Иначе — мужское (с учётом, что бывают «Саша», «Женя» и т. д., где неоднозначность,
    # для такой неоднозначности можно добавить отдельный список исключений)
    # Но для упрощённого варианта говорим, что всё остальное — мужское.
    return "male"


ValueError: Can't find a dictionary for language 'ru'. Installed languages: []. Try installing pymorphy2-dicts-ru package.

In [78]:
combined_df["FirstName"] = combined_df["Name"].apply(get_first_name)

In [79]:
combined_df

,Name,Date,Review,Rating,Review_Length,FirstName
0,Алина,2025-01-31,К выбору квартиры подошла основательно. Сначал...,5,238,Алина
1,Михаил,2025-01-28,Переехали мы год назад в новостройку от компан...,1,752,Михаил
2,Ольга Николина,2025-01-24,Живу пол года в жилом комплексе Пригород Лесно...,5,223,Ольга
3,Евгения Жовниренко,2025-01-21,Категорически не рекомендую самолёт.Все отличн...,1,561,Евгения
4,Мarina,2025-01-20,"Остановилась на несколько ЖК от Самолёта, но в...",5,664,Мarina
...,...,...,...,...,...,...
411,Нонна,2018-03-29,Хочу связаться с Группой Самолет.Но есть опасе...,4,232,Нонна
412,Надежда.,2018-01-19,Купили квартиру в новостройке ЖК Томилино 2018...,4,215,Надежда
413,Татьяна,2017-08-20,Есть вакансия кладовщика свободная,3,34,Татьяна
414,Ксения,2017-01-27,"Отличный застройщик!все во время сдал,квартиру...",5,228,Ксения


In [80]:
combined_df["Gender"] = combined_df["FirstName"].apply(determine_gender)

In [81]:
combined_df

,Name,Date,Review,Rating,Review_Length,FirstName,Gender
0,Алина,2025-01-31,К выбору квартиры подошла основательно. Сначал...,5,238,Алина,female
1,Михаил,2025-01-28,Переехали мы год назад в новостройку от компан...,1,752,Михаил,male
2,Ольга Николина,2025-01-24,Живу пол года в жилом комплексе Пригород Лесно...,5,223,Ольга,female
3,Евгения Жовниренко,2025-01-21,Категорически не рекомендую самолёт.Все отличн...,1,561,Евгения,female
4,Мarina,2025-01-20,"Остановилась на несколько ЖК от Самолёта, но в...",5,664,Мarina,male
...,...,...,...,...,...,...,...
411,Нонна,2018-03-29,Хочу связаться с Группой Самолет.Но есть опасе...,4,232,Нонна,female
412,Надежда.,2018-01-19,Купили квартиру в новостройке ЖК Томилино 2018...,4,215,Надежда,female
413,Татьяна,2017-08-20,Есть вакансия кладовщика свободная,3,34,Татьяна,female
414,Ксения,2017-01-27,"Отличный застройщик!все во время сдал,квартиру...",5,228,Ксения,female


In [84]:
combined_df["Gender"].value_counts()

Gender
male      214
female    202
Name: count, dtype: int64

In [85]:
combined_df

,Name,Date,Review,Rating,Review_Length,FirstName,Gender
0,Алина,2025-01-31,К выбору квартиры подошла основательно. Сначал...,5,238,Алина,female
1,Михаил,2025-01-28,Переехали мы год назад в новостройку от компан...,1,752,Михаил,male
2,Ольга Николина,2025-01-24,Живу пол года в жилом комплексе Пригород Лесно...,5,223,Ольга,female
3,Евгения Жовниренко,2025-01-21,Категорически не рекомендую самолёт.Все отличн...,1,561,Евгения,female
4,Мarina,2025-01-20,"Остановилась на несколько ЖК от Самолёта, но в...",5,664,Мarina,male
...,...,...,...,...,...,...,...
411,Нонна,2018-03-29,Хочу связаться с Группой Самолет.Но есть опасе...,4,232,Нонна,female
412,Надежда.,2018-01-19,Купили квартиру в новостройке ЖК Томилино 2018...,4,215,Надежда,female
413,Татьяна,2017-08-20,Есть вакансия кладовщика свободная,3,34,Татьяна,female
414,Ксения,2017-01-27,"Отличный застройщик!все во время сдал,квартиру...",5,228,Ксения,female


In [83]:
combined_df.to_csv("United_data.csv")

### Добавление данных

In [91]:
combined_df_cutted = combined_df[
    [
        "Name",
        "Date",
        "Review"
    ]
]

In [86]:
sec_data = pd.read_csv("novostroev_reviews.csv")

In [88]:
sec_data["Date"] = pd.to_datetime(sec_data["Date"], format="%d.%m.%Y %H:%M").dt.strftime("%Y-%m-%d")

In [89]:
sec_data

,Name,Date,Review
0,Яна Купчиненко,2024-05-02,"Мы проживаем в жк Алхимово, ни разу не пожалел..."
1,Елена,2021-11-23,У нас знакомые квартиру взяли у этого застройщ...
2,петр,2019-11-15,все хвалебные отзывы - вранье...контора гнилая...
3,Митрий,2020-09-09,"Так это же стандарт такой : ""Прагматик"" называ..."
4,Сергей,2020-11-26,Не выдумывай бро))
...,...,...,...
759,Николай,2024-06-05,Приветствую всех желающих купить квартиру у ГК...
760,Али,2024-02-27,"Отвратительная компания, хуже во всем ( стройк..."
761,Юлия,2024-02-06,Отвратительное и неприятное общение в офисе пр...
762,Екатерина,2024-03-05,При приеме квартиры были недочеты. Исправлять ...


In [92]:
combined_df_cutted

,Name,Date,Review
0,Алина,2025-01-31,К выбору квартиры подошла основательно. Сначал...
1,Михаил,2025-01-28,Переехали мы год назад в новостройку от компан...
2,Ольга Николина,2025-01-24,Живу пол года в жилом комплексе Пригород Лесно...
3,Евгения Жовниренко,2025-01-21,Категорически не рекомендую самолёт.Все отличн...
4,Мarina,2025-01-20,"Остановилась на несколько ЖК от Самолёта, но в..."
...,...,...,...
411,Нонна,2018-03-29,Хочу связаться с Группой Самолет.Но есть опасе...
412,Надежда.,2018-01-19,Купили квартиру в новостройке ЖК Томилино 2018...
413,Татьяна,2017-08-20,Есть вакансия кладовщика свободная
414,Ксения,2017-01-27,"Отличный застройщик!все во время сдал,квартиру..."


In [93]:
massive_data = pd.concat([combined_df_cutted, sec_data], ignore_index=True)

In [95]:
massive_data

,Name,Date,Review
0,Алина,2025-01-31,К выбору квартиры подошла основательно. Сначал...
1,Михаил,2025-01-28,Переехали мы год назад в новостройку от компан...
2,Ольга Николина,2025-01-24,Живу пол года в жилом комплексе Пригород Лесно...
3,Евгения Жовниренко,2025-01-21,Категорически не рекомендую самолёт.Все отличн...
4,Мarina,2025-01-20,"Остановилась на несколько ЖК от Самолёта, но в..."
...,...,...,...
1175,Николай,2024-06-05,Приветствую всех желающих купить квартиру у ГК...
1176,Али,2024-02-27,"Отвратительная компания, хуже во всем ( стройк..."
1177,Юлия,2024-02-06,Отвратительное и неприятное общение в офисе пр...
1178,Екатерина,2024-03-05,При приеме квартиры были недочеты. Исправлять ...


In [97]:
massive_data.to_csv("collected_data.csv")

# SDF

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("merged_4_sites.csv", index_col=0)

In [15]:
df

,Name,Date,Review,Rating,Developer,Site
0,Никита,2025-02-28,Мы живем в арендованной квартире 8 лет. Вернее...,5.0,samolet,novostroy
1,Галина,2025-02-26,При покупке студии в Молжаниново была завышена...,1.0,samolet,novostroy
2,Наталья,2025-02-19,"Удачно переехали в Горки парк, сейчас вот ждем...",5.0,samolet,novostroy
3,Валентина Гвоздилова,2025-02-14,Живем с мужем и сыном в ЖК Большое Путилково у...,5.0,samolet,novostroy
4,Дмитрий Усачин,2025-02-08,Мне комфортно в Новом Внукове от Самолёта. Дом...,5.0,samolet,novostroy
...,...,...,...,...,...,...
1846,Елена197967,2018-06-25,Купили квартиру и очень пожалели. 1. Гранель н...,1.0,granel,otzovik
1847,Алексей Ильичёв,2019-02-18,Ночь 3.51 жена говорит у нас крыша взлетает) К...,1.0,granel,otzovik
1848,Елена7779898,2018-04-11,Решила купить квартиру и сразу начались пробле...,1.0,granel,otzovik
1849,polinavoda,2019-11-06,"Брали квартиру без отделки, поэтому, конечно, ...",5.0,granel,otzovik


In [16]:
df["Review"] = df["Review"].str.replace("Достоинства:", "", regex=False)
df["Review"] = df["Review"].str.replace("Недостатки:", "", regex=False)

In [17]:
df.to_csv("merged_4_sites.csv")

In [18]:
df

,Name,Date,Review,Rating,Developer,Site
0,Никита,2025-02-28,Мы живем в арендованной квартире 8 лет. Вернее...,5.0,samolet,novostroy
1,Галина,2025-02-26,При покупке студии в Молжаниново была завышена...,1.0,samolet,novostroy
2,Наталья,2025-02-19,"Удачно переехали в Горки парк, сейчас вот ждем...",5.0,samolet,novostroy
3,Валентина Гвоздилова,2025-02-14,Живем с мужем и сыном в ЖК Большое Путилково у...,5.0,samolet,novostroy
4,Дмитрий Усачин,2025-02-08,Мне комфортно в Новом Внукове от Самолёта. Дом...,5.0,samolet,novostroy
...,...,...,...,...,...,...
1846,Елена197967,2018-06-25,Купили квартиру и очень пожалели. 1. Гранель н...,1.0,granel,otzovik
1847,Алексей Ильичёв,2019-02-18,Ночь 3.51 жена говорит у нас крыша взлетает) К...,1.0,granel,otzovik
1848,Елена7779898,2018-04-11,Решила купить квартиру и сразу начались пробле...,1.0,granel,otzovik
1849,polinavoda,2019-11-06,"Брали квартиру без отделки, поэтому, конечно, ...",5.0,granel,otzovik


In [19]:
df = pd.read_csv("Сантимент - merged_4_sites.csv", index_col=0)

In [21]:
df.to_csv("collected_data.csv")

In [22]:
df

,Name,Date,Review,Rating,Developer,Site,True_santiment
0,Никита,2025-02-28,Мы живем в арендованной квартире 8 лет. Вернее...,5.0,samolet,novostroy,1
1,Галина,2025-02-26,При покупке студии в Молжаниново была завышена...,1.0,samolet,novostroy,-1
2,Наталья,2025-02-19,"Удачно переехали в Горки парк, сейчас вот ждем...",5.0,samolet,novostroy,1
3,Валентина Гвоздилова,2025-02-14,Живем с мужем и сыном в ЖК Большое Путилково у...,5.0,samolet,novostroy,1
4,Дмитрий Усачин,2025-02-08,Мне комфортно в Новом Внукове от Самолёта. Дом...,5.0,samolet,novostroy,1
...,...,...,...,...,...,...,...
1846,Елена197967,2018-06-25,Купили квартиру и очень пожалели. 1. Гранель н...,1.0,granel,otzovik,-1
1847,Алексей Ильичёв,2019-02-18,Ночь 3.51 жена говорит у нас крыша взлетает) К...,1.0,granel,otzovik,0
1848,Елена7779898,2018-04-11,Решила купить квартиру и сразу начались пробле...,1.0,granel,otzovik,-1
1849,polinavoda,2019-11-06,"Брали квартиру без отделки, поэтому, конечно, ...",5.0,granel,otzovik,1
